In [1]:
"""
Acknowledgement
---------------
1. set your browser download location to your desire location because all files will be downloaded to the download location setted in your browser.
2. create these files -> files_info.json, bot.log, .secret

Thanks -> pythonjar, MariyaSha and some other stackoverflow members...

"""
# Packages & Libraries
import json
from json.decoder import JSONDecodeError
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# Custom imports
from constantsAndVariables import (
    secrets,
    encoding,
    fileName_xpath,
    fileOption_xpath,
    target_file_type,
    fileTypeDate_xpath,
    tracker_file_location,
    download_thread_count,
    download_button_cssSelector,
)
from helperFunctions import (
    login,
    updateLog,
    isDownloaded,
    waitNSeconds,
    downloadFile,
    loadMoreFiles,
    appendFilesInfo,
    initializeWebpage,
    initializeWebDriver,
    waitToFinishDownload,
    getExistingFilesInfo,
)


In [3]:
driver, wait = initializeWebDriver()
login(driver, wait)


WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x00779943+2595139]
	Ordinal0 [0x0070C9F1+2148849]
	Ordinal0 [0x00604528+1066280]
	Ordinal0 [0x00600DB4+1052084]
	Ordinal0 [0x005F63BD+1008573]
	Ordinal0 [0x005F6F7C+1011580]
	Ordinal0 [0x005F65CA+1009098]
	Ordinal0 [0x005F5BC6+1006534]
	Ordinal0 [0x005F4AD0+1002192]
	Ordinal0 [0x005F4FAD+1003437]
	Ordinal0 [0x00605C4A+1072202]
	Ordinal0 [0x0065C19D+1425821]
	Ordinal0 [0x0064B9EC+1358316]
	Ordinal0 [0x0065BAF2+1424114]
	Ordinal0 [0x0064B806+1357830]
	Ordinal0 [0x00626086+1204358]
	Ordinal0 [0x00626F96+1208214]
	GetHandleVerifier [0x0091B232+1658114]
	GetHandleVerifier [0x009D312C+2411516]
	GetHandleVerifier [0x0080F261+560433]
	GetHandleVerifier [0x0080E366+556598]
	Ordinal0 [0x0071286B+2173035]
	Ordinal0 [0x007175F8+2192888]
	Ordinal0 [0x007176E5+2193125]
	Ordinal0 [0x007211FC+2232828]
	BaseThreadInitThunk [0x764B6739+25]
	RtlGetFullPathNam

In [ ]:
initializeWebpage(driver, "https://www.facebook.com/groups/201623576939858/files/")


In [4]:
j, k = 0, 1

# Refresh existing files info every time this cell loads
downloaded_files, tracked_files = getExistingFilesInfo()

# Total files registered in the tracker file(files_info.json)
if tracked_files:
    with open(tracker_file_location, "r", encoding=encoding) as f:
        try:
            # Last registered file id
            registered_files_count = int(json.load(f)["files"][-1]["id"])
        except JSONDecodeError:
            registered_files_count = 0

####################################
# Grabbing initial loadeded target #
####################################
file_option_buttons = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, fileOption_xpath))
)
file_names = wait.until(EC.presence_of_all_elements_located((By.XPATH, fileName_xpath)))
file_types_and_dates = wait.until(
    EC.presence_of_all_elements_located((By.XPATH, fileTypeDate_xpath))
)


for idx, button in enumerate(file_option_buttons):
    try:
        _name = file_names[idx].text
        _type = file_types_and_dates[j].text
        _date = file_types_and_dates[k].text

        # as 'file_types_and_dates' contains both file-type and date
        j += 2
        k += 2

        # Scrolling after every 15th check to load more files (as by default 15 files loads per scroll)
        if (idx + 1) % 15 == 0:
            [file_option_buttons, file_names, file_types_and_dates] = loadMoreFiles(
                driver,
                [file_option_buttons, file_names, file_types_and_dates],
                [fileOption_xpath, fileName_xpath, fileTypeDate_xpath],
            )

            updateLog("\nTotal Loaded Files: {}\n".format(len(file_option_buttons)))

        """
        Download pdf files only
        Cannot check this before the scroll because
        here is a possibility to have more than one page worth of non-pdf files
        
        """
        if not _type == target_file_type.upper():
            updateLog(
                '\n😪 Skipping: "{}", 🤔 Reason: FILE_TYPE: "{}"\n'.format(_name, _type)
            )
            continue

        #################################
        # Skip already downloaded files #
        #################################
        if isDownloaded(_name, _date, downloaded_files, tracked_files):
            updateLog('😪 Skipping: "{}", 🤔 Reason: ALREADY_DOWNLOADED'.format(_name))
            continue

        #############################################################
        # If everything is OK then initiating file download         #
        #############################################################
        # this was giving me -> ElementClickInterceptedException -> #
        # wait.until(EC.element_to_be_clickable(button)).click()    #
        #############################################################

        driver.execute_script(
            "arguments[0].click();", button
        )  # Clicking the download button

        waitNSeconds()

        try:  # Try to download the file
            registered_files_count = downloadFile(
                driver, wait, download_button_cssSelector, registered_files_count
            )

            """
            Waiting until the file gets downloaded completely
            We have to wait for the file to be downloaded, if we continue without it there maybe some files
            that couldn't be downloaded on time (before the link expire) and later on those files can't be resumed
            
            """
            download_time = waitToFinishDownload(secrets[2], registered_files_count)

            # Waiting 600s before stoping the Bot for slow/no internet 😟
            if download_time == -1:
                break

            # Updating tracker file after a successful file download 😎
            appendFilesInfo(
                {
                    "id": registered_files_count,
                    "type": _type,
                    "name": _name,
                    "uploaded_date": _date,
                }
            )

            """ Waiting at least 2s before every download request for safety 😅 """

        except:
            updateLog(
                "\n*** Warning: No download link found at {} ***\n".format(idx + 1)
            )

        # Updating info after a successful file download 😎
        updateLog('✔ DONE: "{}", "{}", "{}"'.format(_type, _date, _name))

        break

    except Exception as e:
        updateLog("\n*** ERROR at {}, date: {} ---> {} ***\n".format(_name, _date, e))

# driver.close() # not closing because I had to run this cell multiple times during test
updateLog(
    "\n👏👏👏 Completed 👏👏👏\n\nFiles downloaded: {}\n".format(registered_files_count)
)
updateLog(
    "Go ahead and give this https://github.com/ShahriarDhruvo/Bots a star, thanks\t-- SED"
)


WebDriverException: Message: chrome not reachable
  (Session info: chrome=99.0.4844.51)
Stacktrace:
Backtrace:
	Ordinal0 [0x00779943+2595139]
	Ordinal0 [0x0070C9F1+2148849]
	Ordinal0 [0x006043F0+1065968]
	Ordinal0 [0x005F87C2+1017794]
	Ordinal0 [0x005F8FF8+1019896]
	Ordinal0 [0x005FA892+1026194]
	Ordinal0 [0x005F4219+999961]
	Ordinal0 [0x00605860+1071200]
	Ordinal0 [0x0065B2D2+1422034]
	Ordinal0 [0x0064B806+1357830]
	Ordinal0 [0x00626086+1204358]
	Ordinal0 [0x00626F96+1208214]
	GetHandleVerifier [0x0091B232+1658114]
	GetHandleVerifier [0x009D312C+2411516]
	GetHandleVerifier [0x0080F261+560433]
	GetHandleVerifier [0x0080E366+556598]
	Ordinal0 [0x0071286B+2173035]
	Ordinal0 [0x007175F8+2192888]
	Ordinal0 [0x007176E5+2193125]
	Ordinal0 [0x007211FC+2232828]
	BaseThreadInitThunk [0x764B6739+25]
	RtlGetFullPathName_UEx [0x770A8E7F+1215]
	RtlGetFullPathName_UEx [0x770A8E4D+1165]
